In [1]:
import pandas as pd
from datetime import datetime
from functions_load_and_transform import player_list, scoringDF, lastWeek, thisWeek

thisWeek = 20

week_view_list = list(range(1, thisWeek))
week_view_list.append("Summe")

scoringDF = scoringDF.drop("Game Nr.", axis=1)
scoringDF = scoringDF.drop("Winner", axis=1)

weekly_group = scoringDF.groupby("Week").sum()
week_view = weekly_group.transpose()#
week_view["Summe"] = week_view.sum(axis=1)
week_view


week_view = week_view[week_view_list]
week_view.columns = week_view.columns.map(str)

po_vorab_points = pd.read_csv("data/playoffResults.csv").transpose().rename(columns={0:"Playoff Vorab Punkte"})
po_vorab_points["Regular Season Punkte"] = week_view["Summe"]
po_vorab_points["Gesamtpunkte"] = po_vorab_points["Regular Season Punkte"]+po_vorab_points["Playoff Vorab Punkte"]
po_vorab_points.to_csv("data/end_reg_Season.csv", index=False)

reg_SeasonDF = pd.read_csv("data/end_reg_Season.csv")
betsDF = pd.read_csv("data/bets_2024.csv")
betsDF = betsDF.loc[betsDF["Week"]>=19].copy()
resultsDF = pd.read_csv("data/results.csv")
resultsDF = resultsDF.loc[resultsDF["Week"]>=19].copy()

scoringDF = pd.concat([betsDF[player_list], resultsDF[["Winner", "Week", "Game Nr."]]], axis=1)
for player in player_list:
    # scoringDF[f"score_{player}"] = scoringDF.apply(lambda row: 1 if row[player] == row["Winner"] else 0, axis=1)
    scoringDF[f"score_{player}"] = scoringDF.apply(lambda row: 2 if row[player] == row["Winner"] else 0, axis=1)
scoringDF
scoringDF.to_csv("data/playoff_scores.csv", index=False)


playoffDF = pd.DataFrame(columns=["Players", "Gesamtpunkte", "Regular Season Points", "Wildcard Games", "Divisional Round", "Conference Finals", "Superbowl"])
playoffDF["Players"] = player_list
playoffDF = playoffDF.fillna(0)
playoffDF["Regular Season Points"] = list(reg_SeasonDF["Gesamtpunkte"])
for player in player_list:
    playoffDF.loc[playoffDF["Players"]==player, "Wildcard Games"] = scoringDF.loc[scoringDF["Week"]==19, f"score_{player}"].sum()
    playoffDF.loc[playoffDF["Players"]==player, "Divisional Round"] = scoringDF.loc[scoringDF["Week"]==20, f"score_{player}"].sum()
playoffDF["Gesamtpunkte"] = playoffDF["Regular Season Points"] + playoffDF["Wildcard Games"] + playoffDF["Divisional Round"] +playoffDF["Conference Finals"] + playoffDF["Superbowl"]
playoffDF.to_csv("data/playoff_view.csv", index=False)
playoffDF

,Players,Gesamtpunkte,Regular Season Points,Wildcard Games,Divisional Round,Conference Finals,Superbowl
0,Alex,215,203,8,4,0,0
1,Alina,196,184,10,2,0,0
2,Evelyn,223,213,6,4,0,0
3,Christopher,211,197,8,6,0,0
4,Ludwig,89,89,0,0,0,0
5,Manu,118,116,0,2,0,0
6,Natalie,189,183,6,0,0,0
7,Nikolai,48,48,0,0,0,0
8,Sebastian,183,173,6,4,0,0
9,Vero,192,180,10,2,0,0


,Week,Game Nr.,Home Team,Away Team,Score Home,Score Guest,Winner,Looser
